In [18]:
import requests
import re
import json
import time
from datetime import datetime
import csv

In [ ]:
def get_video_info(bv_id):
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36 Edg/143.0.0.0",
        }
    api_url = f'https://api.bilibili.com/x/web-interface/view?bvid={bv_id}'
    # 打印本次要获取的bvid，用于错误时确认
    print(f"正在进行爬取uid为：{bv_id}的UP主的粉丝数量与作品总数")
    print(f"==========本次获取数据的视频BV号为：{bv_id}==========")
    print(f"url为：{api_url}")
    # https://api.bilibili.com/x/web-interface/view?BV1n24y1D75V
    video_info = requests.get(url=api_url, headers=headers)
    video_info_json = json.loads(video_info.text)
    
    # 创建存放的字典
    info_dict = {}
    # 信息解读
    # https://zhuanlan.zhihu.com/p/618885790
    # 视频bvid，即bv号
    bvid = video_info_json['data']['bvid']
    info_dict['bvid'] = bvid
    # 视频aid，即av号
    aid = video_info_json['data']['aid']
    info_dict['aid'] = aid
    # 视频cid，用于获取弹幕信息
    cid = video_info_json['data']['cid']
    info_dict['cid'] = cid
    # 作者id
    mid = video_info_json['data']['owner']['mid']
    info_dict['mid'] = mid
    # up主昵称
    name = video_info_json['data']['owner']['name']
    info_dict['name'] = name
    # 视频标题
    title = video_info_json['data']['title']
    info_dict['title'] = title
    # 视频标签
    tname = video_info_json['data']['tname']
    info_dict['tname'] = tname
    # 视频发布时间戳
    pubdate = video_info_json['data']['pubdate']
    # 转化时间戳
    pub_datatime = datetime.fromtimestamp(pubdate)
    # 整体格式
    pub_datatime_strf = pub_datatime.strftime('%Y-%m-%d %H:%M:%S')
    # 日期
    date = re.search(r"(\d{4}-\d{1,2}-\d{1,2})", pub_datatime_strf)
    info_dict['pub_date'] = date.group()
    # 时间
    pub_time = re.search(r"(\d{1,2}:\d{1,2}:\d{1,2})", pub_datatime_strf)
    info_dict['pub_time'] = pub_time.group()
    # 视频创建时间戳
    # ctime = info['ctime']
    # 视频简介
    desc = video_info_json['data']['desc']
    info_dict['desc'] = desc
    # 视频播放量
    view = video_info_json['data']['stat']['view']
    info_dict['view'] = view
    # 点赞数
    like = video_info_json['data']['stat']['like']
    info_dict['like'] = like
    # 投币数
    coin = video_info_json['data']['stat']['coin']
    info_dict['coin'] = coin
    # 收藏数
    favorite = video_info_json['data']['stat']['favorite']
    info_dict['favorite'] = favorite
    # 分享数
    share = video_info_json['data']['stat']['share']
    info_dict['share'] = share
    # 评论数
    repiy = video_info_json['data']['stat']['reply']
    info_dict['reply'] = repiy
    # 视频弹幕数量
    danmaku = video_info_json['data']['stat']['danmaku']
    info_dict['danmaku'] = danmaku
 
    print(f'=========={bv_id} 的视频基本信息已成功获取==========')
 
    # 发布作品时的动态
    # dynamic = info['dynamic']
    print('正在等待，以防访问过于频繁\n')
    time.sleep(3)
 
    return info_dict


In [20]:
def write_video_info_to_csv(info_dict_list, csv_path="video_info.csv"):
    """
    批量写入视频信息到CSV
    :param info_dict_list: 包含多个info_dict的列表
    :param csv_path: 输出CSV路径
    """
    if not info_dict_list:
        print("无视频信息可写入！")
        return
    
    # 定义CSV表头（和info_dict的key对应，顺序可自定义）
    headers = [
        'bvid', 'aid', 'cid', 'mid', 'name', 'title', 'tname',
        'pub_date', 'pub_time', 'desc', 'view', 'like', 'coin',
        'favorite', 'share', 'reply', 'danmaku'
    ]
    
    # 写入CSV（追加模式：a，避免覆盖已有数据；newline=""避免空行）
    with open(csv_path, "a", encoding="utf-8-sig", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        
        # 如果文件是空的，先写入表头
        import os
        if os.path.getsize(csv_path) == 0:
            writer.writeheader()
        
        # 批量写入每条视频信息
        for info in info_dict_list:
            writer.writerow(info)
    print(f"==========已将{len(info_dict_list)}条视频信息写入CSV：{csv_path}==========")

In [ ]:
csv_file_path = "bvid_list.csv"
with open(csv_file_path, "r", encoding="utf-8-sig") as f:
    # 创建CSV读取器
    csv_reader = csv.reader(f)
    # 步骤2：跳过表头（第一行是"BVID"，不需要遍历）
    next(csv_reader)  # 执行这行后，读取器会从第二行开始
    
    for row in csv_reader:
        # row是一个列表，比如第二行是["BV1234567890"]，取第一个元素就是BVID
        bvid = row[0]
        # 打印/处理每个BVID（替换成你的业务逻辑，比如访问视频链接）
        print("当前遍历到的BVID：", bvid)

        # 调用函数获取视频信息
        try:
            video_info = get_video_info(bvid)
            if video_info is not None:  # 先判断爬取是否成功，避免传入None
                write_video_info_to_csv([video_info])  # 用[]把单个字典包装成列表
            else:
                print(f"{bvid}爬取失败，跳过写入")
        except :
            print(f"{bvid} 爬取过程中出现错误，跳过写入")

当前遍历到的BVID： BV1q7x6zWEG7
正在进行爬取uid为：BV1q7x6zWEG7的UP主的粉丝数量与作品总数
==========本次获取数据的视频BV号为：BV1q7x6zWEG7==========
url为：https://api.bilibili.com/x/web-interface/view?bvid=BV1q7x6zWEG7
==========BV1q7x6zWEG7 的视频基本信息已成功获取==========
正在等待，以防访问过于频繁

==========已将1条视频信息写入CSV：video_info.csv==========
当前遍历到的BVID： BV1Ca41177oT
正在进行爬取uid为：BV1Ca41177oT的UP主的粉丝数量与作品总数
==========本次获取数据的视频BV号为：BV1Ca41177oT==========
url为：https://api.bilibili.com/x/web-interface/view?bvid=BV1Ca41177oT
==========BV1Ca41177oT 的视频基本信息已成功获取==========
正在等待，以防访问过于频繁

==========已将1条视频信息写入CSV：video_info.csv==========
当前遍历到的BVID： BV1zBDAYgEoy
正在进行爬取uid为：BV1zBDAYgEoy的UP主的粉丝数量与作品总数
==========本次获取数据的视频BV号为：BV1zBDAYgEoy==========
url为：https://api.bilibili.com/x/web-interface/view?bvid=BV1zBDAYgEoy
==========BV1zBDAYgEoy 的视频基本信息已成功获取==========
正在等待，以防访问过于频繁

==========已将1条视频信息写入CSV：video_info.csv==========
当前遍历到的BVID： BV1MYmgBtEGr
正在进行爬取uid为：BV1MYmgBtEGr的UP主的粉丝数量与作品总数
==========本次获取数据的视频BV号为：BV1MYmgBtEGr==========
url为：https://api.bilibi